![Astrofisica Computacional](../logo.PNG)

---
## 01. Algoritmo de Barnes-Hut

Eduard Larrañaga (ealarranaga@unal.edu.co)

---


### Resumen

Se presentan el método de Barnes-Hut para describir la dinámica de un sistema gravitacional de N-cuerpos.


---

## El Algoritmo Barnes-Hut

El algoritmo Barnes-Hut (descrito incialmente por Josh Barnes y Piet Hut) presenta un método de solución del problema gravitacional de N-cuerpos que logra reducir el orden $\mathcal{O} (N^2)$ de los métodos de suma directa (fuerza bruta) a un orden $\mathcal{O} (N \log N)$.

La idea central de este algoritmo corresponde a dividir el espacio físico donde se ubican las particulas en celdas cúbicas para formar un **octo-árbol** (si el espacio es 3D) o en celdas cuadradas para formar un **cuadra-árbol** (si el espacio es 2D). Un cuadra-árbol (*quad-tree*) es similar a un arbol binario pero cada nodo posee 4 *hijos* (algunos de los cuales podrán estar vacios). En este modelo, cada nodo representa una región del espacio.

Si existe un conjunto de particulas distantes y muy juntas, estas se pueden agrupar en la forma de un solo objeto con la masa del conjunto y localizado en una celda ubicada en el centro de masa del grupo. Esta descripción corresponde a describir el conjunto de partículas mediante una expansión multipolar de bajo orden (orden de monopolo). Al realizar esta aproximación, se reduce dramáticamente el número cálculos de interacciones.

--- 
### El Algoritmo en 2-Dimensiones 

Considerese un conjunto de N-cuerpos moviendose en el espacio 2-dimensional bajo su interacción gravitacional mutua. En la descripción de Barnes-Hut, se divide recursivamente el conjunto de N-cuerpos en grupos almacenando su información en un cuadra-árbol formado por las celdas cuadradas (cuadrantes) en las que se va dividiendo el espacio físico. De esta forma, cada **nodo** del árbol corresponde auna región del espacio.

El nodo inicial (root) representa todo el espacio físico y de él se desprenderán cuatro nodos que representan los cuatro cuadrantes en los que se divide ese espacio. Luego cada nodo de este segundo orden se sigue dividiendo recursivamente en cuadrantes hasta el momento en el que cada uno de ellos contenga una sola partícula o este vacio. En la siguiente figura se puede observar el árbol generado para un espacio 2-dimensional con 8 partículas:

![](BarnesHutTree.001.jpeg)

De esta manera, existen tres tipos de nodos en el árbol:

- Nodos externos : No posee nodos *hijos*, es decir no se sub-divide. En estos nodos se encuentra una partícula o esta vacio. En caso de incluir en el una partícula, el nodo debe almacenar la información actual de la masa, posición y velocidad de la partícula.

- Nodos internos: Cada nodo interno posee sub-divisiones y representa al grupo completo de particulas que se localizan en los nodos hijos que posee. El nodo interno almacena la información de la masa total de las particulas debajo de él junto con la posición del centro de masa correspondiente.

- Nodos Vacios: Representan un cuadrante vacio.

El árbol que describe el sistema físico debe ser actualizado en cada uno de los pasos del proceso de integración.

**Construcción del Árbol**

Para la construcción del árbol se insertan los cuerpos uno a uno siguiendo este procedimiento de recursión:

1. Si el nodo en el que se quiere poner no contiene cuerpos, se coloca el cuerpo allí y se actualiza el valor de la masa y el centro de masa asociado al nodo.
2. Si el nodo es un *nodo interno* (i.e. ya tiene nodos hijo) se actualiza el valor de la masa total y el centro de masa, para luego proceder a insertar el cuerpo recursivamente en el cuadrante apropiado.
3. Si el nodo es un *nodo externo* (i.e. ya contiene una sola particula), entonces los dos cuerpos estarán en la misma region. Se debe sub-dividir el nodo en cuatro hijos y luego insertar las dos partículas en el cuadrante apropiado recursivamente. En caso en el que las dos particulas vuelvan a ubicarse en el mismo sub-cuadrante, es necesario volver a sub-dividir ese cuadrante y continuar con la recursión. Finalmente, debe actualizarse la masa total y el centro de masa. 

#### El Cálculo de la Fuerza Gravitacional

Para calcular la fuerza neta sobre una partículas específica $N_k$ del sistema en un instante de tiempo dado es necesario recorrer todos los nodos del árbol, comenzando desde el *root*. Si el centro de masa de uno de los nodos internos está lo suficientemente lejos del cuerpo $N_k$,todos los cuerpos contenidos por el nodo serán tratados como una sola partícula cuya masa será la masa total del conjunto y su ubicación estará dada por la posición del centro de masa respectivo. Por otro lado, si el nodo interno esta suficiente mente cerca de $N_k$, es necesario repetir el proceso con cada uno de sus nodos hijo.

El criterio de cercanía utilizado se realiza comparando la razón $\frac{s}{d}$ (con $s$ el lado del cuadrante descrito por el nodo interno y $d$ la distancia desde $N_k$ hasta el centro de masa almacenado en el nodo interno) con un valor de corte $\theta$ que define la precisión del algoritmo. De esta forma, si $\frac{s}{d} <\theta$ se dice que el nodo esta suficientemente lejos de $N_k$, mientras que si $\frac{s}{d} >\theta$, el nodo esta demasiado cerca y deben considerarse sus nodos hijo.

**Nota**: Si se toma $\theta =0$, ningún nodo interno se considera como un solo cuerpo y el algoritmo de Barnes-Hut se convierte en el algoritmo de suma-directa (fuerza bruta) considerado en clases anteriores.

El proceso para calcular la fuerza neta sobre un cuerpo **b** inicia desde el nodo root y continua con la siguiente recursión sobre todos los nodos:

1. Si el nodo actual es un *nodo externo* y no corresponde al cuerpo **b** (para evitar auto-gravedad), se calcula la fuerza ejercida por el nodo sobre **b** y se suma a la fuerza neta.

2. Si el nodo actual NO es un *nodo externo* se calcula la razón $\frac{s}{d}$. 
 - Si $\frac{s}{d}<\theta$, el nodo puede tratarse como un solo cuerpo. Se calcula la fuerza que este ejerce sobre el cuerpo **b** y se adiciona este valor a la fuerza neta.
 
 - Si $\frac{s}{d}>\theta$, debe procederse ecursivamente sobre cada uno de los hijos del nodo actual.